In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Masking, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

2024-04-30 19:44:17.054684: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-30 19:44:17.141060: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 19:44:18.020657: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np
import cv2  # OpenCVライブラリ
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import os
import sys
import pathlib
from pathlib import Path

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
current_file_path = pathlib.Path(os.getcwd())
parent_dir = current_file_path.parent

In [4]:
import fluid_intake_estimate
from fluid_intake_estimate import variable_data_set
from fluid_intake_estimate import cnn

/home/s2/Desktop/washino/swallowing
/home/s2/Desktop/washino/swallowing


In [10]:
directory_path = parent_dir / 'dataset'
path = directory_path / 'washino'

In [12]:
csv_path = path / 'washino.csv'
data = variable_data_set.VariableDataSet(250)
data.csv_to_dataset(path, csv_path, 0)
model = cnn.CNN()

2024-04-30 20:11:53.069381: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22052 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


In [15]:
model_checkpoint = ModelCheckpoint(
    filepath='cnn_20240430_model_best.keras',  # 保存するファイルのパス
    monitor='val_mean_absolute_error',        # 監視する値
    save_best_only=True,       # 最良のモデルのみを保存
    verbose=1,                 # 進行状況のメッセージ表示
    mode='min'                 # 「減少」を監視
)

model.training(data.X, data.y, 50, 32, model_checkpoint = model_checkpoint)

Epoch 1/50
7/8 [=========================>....] - ETA: 0s - loss: 0.0042 - mean_absolute_error: 0.0458 - mean_squared_error: 0.0042
Epoch 1: val_mean_absolute_error improved from inf to 0.25070, saving model to cnn_20240430_model_best.keras
8/8 [==============================] - 2s 291ms/step - loss: 0.0043 - mean_absolute_error: 0.0461 - mean_squared_error: 0.0043 - val_loss: 0.0882 - val_mean_absolute_error: 0.2507 - val_mean_squared_error: 0.0882
Epoch 2/50
7/8 [=========================>....] - ETA: 0s - loss: 0.0035 - mean_absolute_error: 0.0432 - mean_squared_error: 0.0035
Epoch 2: val_mean_absolute_error improved from 0.25070 to 0.22901, saving model to cnn_20240430_model_best.keras
8/8 [==============================] - 1s 176ms/step - loss: 0.0035 - mean_absolute_error: 0.0430 - mean_squared_error: 0.0035 - val_loss: 0.0799 - val_mean_absolute_error: 0.2290 - val_mean_squared_error: 0.0799
Epoch 3/50
7/8 [=========================>....] - ETA: 0s - loss: 0.0032 - mean_absolute

Epoch 21/50
7/8 [=========================>....] - ETA: 0s - loss: 0.0032 - mean_absolute_error: 0.0422 - mean_squared_error: 0.0032
Epoch 21: val_mean_absolute_error did not improve from 0.19360
8/8 [==============================] - 1s 166ms/step - loss: 0.0032 - mean_absolute_error: 0.0423 - mean_squared_error: 0.0032 - val_loss: 0.0690 - val_mean_absolute_error: 0.2113 - val_mean_squared_error: 0.0690
Epoch 22/50
7/8 [=========================>....] - ETA: 0s - loss: 0.0028 - mean_absolute_error: 0.0413 - mean_squared_error: 0.0028
Epoch 22: val_mean_absolute_error did not improve from 0.19360
8/8 [==============================] - 1s 167ms/step - loss: 0.0028 - mean_absolute_error: 0.0414 - mean_squared_error: 0.0028 - val_loss: 0.1143 - val_mean_absolute_error: 0.2893 - val_mean_squared_error: 0.1143
Epoch 23/50
7/8 [=========================>....] - ETA: 0s - loss: 0.0035 - mean_absolute_error: 0.0419 - mean_squared_error: 0.0035
Epoch 23: val_mean_absolute_error improved from 0

Epoch 41/50
7/8 [=========================>....] - ETA: 0s - loss: 0.0015 - mean_absolute_error: 0.0244 - mean_squared_error: 0.0015
Epoch 41: val_mean_absolute_error did not improve from 0.17561
8/8 [==============================] - 1s 174ms/step - loss: 0.0015 - mean_absolute_error: 0.0244 - mean_squared_error: 0.0015 - val_loss: 0.0915 - val_mean_absolute_error: 0.2536 - val_mean_squared_error: 0.0915
Epoch 42/50
7/8 [=========================>....] - ETA: 0s - loss: 0.0014 - mean_absolute_error: 0.0237 - mean_squared_error: 0.0014
Epoch 42: val_mean_absolute_error did not improve from 0.17561
8/8 [==============================] - 1s 167ms/step - loss: 0.0015 - mean_absolute_error: 0.0242 - mean_squared_error: 0.0015 - val_loss: 0.1038 - val_mean_absolute_error: 0.2869 - val_mean_squared_error: 0.1038
Epoch 43/50
7/8 [=========================>....] - ETA: 0s - loss: 0.0124 - mean_absolute_error: 0.0803 - mean_squared_error: 0.0124
Epoch 43: val_mean_absolute_error did not improve

In [8]:
directory = path

# ファイル名の変更をループで実行
for i in range(150, 201):
    old_file = f'20240422_{i}.wav'
    new_file = f'20240424_{i}.wav'

    # 完全なファイルパスを構築
    old_file_path = os.path.join(directory, old_file)
    new_file_path = os.path.join(directory, new_file)
    
    # ファイル名を変更（リネーム）
    if os.path.exists(old_file_path):  # ファイルが存在するか確認
        os.rename(old_file_path, new_file_path)
        print(f'Renamed {old_file} to {new_file}')
    else:
        print(f'{old_file} not found')

Renamed 20240422_150.wav to 20240424_150.wav
Renamed 20240422_151.wav to 20240424_151.wav
Renamed 20240422_152.wav to 20240424_152.wav
Renamed 20240422_153.wav to 20240424_153.wav
Renamed 20240422_154.wav to 20240424_154.wav
Renamed 20240422_155.wav to 20240424_155.wav
Renamed 20240422_156.wav to 20240424_156.wav
Renamed 20240422_157.wav to 20240424_157.wav
Renamed 20240422_158.wav to 20240424_158.wav
Renamed 20240422_159.wav to 20240424_159.wav
Renamed 20240422_160.wav to 20240424_160.wav
Renamed 20240422_161.wav to 20240424_161.wav
Renamed 20240422_162.wav to 20240424_162.wav
Renamed 20240422_163.wav to 20240424_163.wav
Renamed 20240422_164.wav to 20240424_164.wav
Renamed 20240422_165.wav to 20240424_165.wav
Renamed 20240422_166.wav to 20240424_166.wav
Renamed 20240422_167.wav to 20240424_167.wav
Renamed 20240422_168.wav to 20240424_168.wav
Renamed 20240422_169.wav to 20240424_169.wav
Renamed 20240422_170.wav to 20240424_170.wav
Renamed 20240422_171.wav to 20240424_171.wav
Renamed 20